# Processamento dados

## SETUP

### import

In [4]:
import os
import json
import pandas as pd

### Mount Drive

In [27]:
# drive.mount('/content/drive')
# from google.colab import drive

### Constants

In [9]:
BASE_PATH = '/home/lucas-nunes/workspace/Postech/challenges/5_data/data/'
BRONZE_PATH = os.path.join(BASE_PATH, 'bronze')
SILVER_PATH = os.path.join(BASE_PATH, 'silver')

FILENAME_APPLICATIONS = 'applicants.json'
FILENAME_PROSPECTS = 'prospects.json'
FILENAME_JOBS = 'vagas.json'

INPUT_FILE_PATH_APPLICATIONS = os.path.join(BRONZE_PATH, FILENAME_APPLICATIONS)
INPUT_FILE_PATH_PROSPECTS = os.path.join(BRONZE_PATH, FILENAME_PROSPECTS)
INPUT_FILE_PATH_JOBS = os.path.join(BRONZE_PATH, FILENAME_JOBS)

if not os.path.exists(SILVER_PATH): os.makedirs(SILVER_PATH, exist_ok=True)

### VALUE DICTS

In [ ]:
DICT_HEADER_OBESITY_TRANSLATION = {
  'gender': "Gênero",
  'age': "Idade",
  'height': "Altura em metros",
  'weight': "Peso em kgs",
  'family_history': "Algum membro da família sofreu ou sofre de excesso de peso?",
  'favc': "Você come alimentos altamente calóricos com frequência?",
  'fcvc': "Você costuma comer vegetais nas suas refeições?",
  'ncp': "Quantas refeições principais você faz diariamente?",
  'caec': "Você come alguma coisa entre as refeições?",
  'smoke': "Você fuma?",
  'ch2o': "Quanta água você bebe diariamente?",
  'scc': "Você monitora as calorias que ingere diariamente?",
  'faf': "Com que frequência você pratica atividade física?",
  'tue': "Quanto tempo você usa dispositivos tecnológicos como celular, videogame, televisão, computador e outros?",
  'calc': "Com que frequência você bebe álcool?",
  'mtrans': "Qual meio de transporte você costuma usar?",
  'obesity': "Nível de obesidade"
}

DICT_VALUE_OBESITY_TRANSLATION = {
  "Female": "Feminino",
  "Male": "Masculino",
  "yes": "Sim",
  "no": "Não",
  "Sometimes": "Às vezes",
  "Frequently": "Frequentemente",
  "Always": "Sempre",
  "Public_Transportation": "Transporte público",
  "Walking": "Caminhada",
  "Automobile": "Automóvel",
  "Motorbike": "Motocicleta",
  "Bike": "Bicicleta",
  "Normal_Weight": "Peso normal",
  "Overweight_Level_I": "Sobrepeso Nível I",
  "Overweight_Level_II": "Sobrepeso Nível II",
  "Obesity_Type_I": "Obesidade Tipo I",
  "Obesity_Type_II": "Obesidade Tipo II",
  "Obesity_Type_III": "Obesidade Tipo III",
  "Insufficient_Weight": "Peso insuficiente"
}
# a
DICT_FREQUENCY_OBESITY = {
  'no': 0,
  'sometimes': 1,
  'frequently': 2,
  'always': 3
}

DICT_LEVEL_OBESITY = {
  'insufficient_weight': 0,
  'normal_weight': 1,
  'overweight_level_i': 2,
  'overweight_level_ii': 3,
  'obesity_type_i': 4,
  'obesity_type_ii': 5,
  'obesity_type_iii': 6
}

DICT_YES_NO_TO_BOOL = {
  'yes': True,
  'no': False
}

## Read

### Read Functions

In [10]:
def read_json(filename, encoding: str = 'utf-8'):
    with open(filename, 'r+', encoding=encoding) as file:
        value = json.load(file)
    return value

### Read JSON

In [11]:
json_applications = read_json(INPUT_FILE_PATH_APPLICATIONS)
json_prospects = read_json(INPUT_FILE_PATH_PROSPECTS)
json_jobs = read_json(INPUT_FILE_PATH_JOBS)

### Generate Samples

In [20]:
sample_amount = 3


def generate_sample(dict_input, sample_amount: int = 3):
    dict_sample = {}

    count = 0

    for key, value in dict_input.items():
        dict_sample[key] = value
        count += 1
        if count >= sample_amount:
            break
    return dict_sample

dict_sample_applications = generate_sample(json_applications)
dict_sample_prospects = generate_sample(json_prospects)
dict_sample_jobs = generate_sample(json_jobs)

## Process

### Processing functions

In [31]:
def convert_to_int(value):
    value = int(round(float(value), 0))
    return value


def convert_to_bool(value):
    if value == 'yes':
        return True
    elif value == 'no':
        return False
    else:
        raise KeyError(f'item: {value} is not "yes" or "no"')

    
def convert_dict_to_int(value, dict_reference):
    value = str(value).lower()
    if value in dict_reference:
        return dict_reference[value]
    else:
        raise KeyError(f'value: {value} is not one of the keys: {list(dict_reference.keys())}')


def rename_column_to_lower(df):
    dict_values = {value:str(value).strip().lower() for value in df.columns.values}
    df.rename(columns=dict_values, inplace=True)
    return df


def process_all_values(df):
    df['gender'] = df['gender'].astype(str)
    df['age'] = df['age'].T.apply(convert_to_int)
    df['height'] = df['height'].astype(float)
    df['weight'] = df['weight'].astype(float)
    df['family_history'] = df['family_history'].T.apply(convert_dict_to_int, dict_reference=DICT_YES_NO_TO_BOOL)
    df['favc'] = df['favc'].T.apply(convert_dict_to_int, dict_reference=DICT_YES_NO_TO_BOOL)
    df['fcvc'] = df['fcvc'].astype(float)
    df['ncp'] = df['ncp'].T.apply(float)
    df['caec'] = df['caec'].T.apply(convert_dict_to_int, dict_reference=DICT_FREQUENCY_OBESITY)
    df['smoke'] = df['smoke'].T.apply(convert_dict_to_int, dict_reference=DICT_YES_NO_TO_BOOL)
    df['ch2o'] = df['ch2o'].T.apply(float)
    df['scc'] = df['scc'].T.apply(convert_dict_to_int, dict_reference=DICT_YES_NO_TO_BOOL)
    df['faf'] = df['faf'].T.apply(float)
    df['tue'] = df['tue'].T.apply(float)
    df['calc'] = df['calc'].T.apply(convert_dict_to_int, dict_reference=DICT_FREQUENCY_OBESITY)
    df['mtrans'] = df['mtrans'].astype(str)
    df['obesity'] = df['obesity'].T.apply(convert_dict_to_int, dict_reference=DICT_LEVEL_OBESITY)
    return df

def translate_value_fields(df):
    df.replace(DICT_VALUE_OBESITY_TRANSLATION, inplace=True)
    return df

### Process Execution

#### Process Applications dataset

In [ ]:
def parser_values_from_json(dict_json):
    list_records = []
    for job_id, dict_value in dict_json.items():
        dict_processed = dict(job_id=job_id)
        for key, value in dict_value.items():
            if isinstance(value, dict):
                dict_processed.update(**value)
            else:
                dict_processed[key] = value
        list_records.append(dict_processed)
    return list_records




#json_prospects
#json_jobs

,job_id,telefone_recado,telefone,objetivo_profissional,data_criacao,inserido_por,email,local,sabendo_de_nos_por,data_atualizacao,...,nivel_profissional,nivel_academico,nivel_ingles,nivel_espanhol,outro_idioma,cv_pt,cv_en,instituicao_ensino_superior,cursos,ano_conclusao
0,31000,,(11) 97048-2708,,10-11-2021 07:29:49,Luna Correia,carolina_aparecida@gmail.com,,,10-11-2021 07:29:49,...,,,,,-,assistente administrativo\n\n\nsantosbatista\n...,,NaN,NaN,NaN
1,31001,,(11) 93723-4396,Analista Administrativo,10-11-2021 08:56:16,Laura Pacheco,eduardo_rios@hotmail.com,"São Paulo, São Paulo",Outros,11-11-2021 11:10:31,...,,Ensino Superior Incompleto,Nenhum,Nenhum,-,formação acadêmica\nensino médio (2º grau) em ...,,NaN,NaN,NaN
2,31002,,(11) 92399-9824,Administrativo | Financeiro,10-11-2021 09:01:00,Laura Pacheco,pedro_henrique_carvalho@gmail.com,"São Paulo, São Paulo",Anúncio,10-11-2021 11:42:36,...,,Ensino Superior Completo,Intermediário,Básico,Português - Fluente,objetivo: área administrativa | financeira\n\n...,,,Administração de Empresas,2012


In [40]:
list_records_applications = parser_values_from_json(json_applications) # json_applications # dict_sample_applications
# list_records_applications = parser_values_from_json(dict_sample_applications)
# list_records_applications = parser_values_from_json(dict_sample_applications)

df_applications = pd.DataFrame.from_records(list_records_applications)

In [41]:
df_applications

,job_id,telefone_recado,telefone,objetivo_profissional,data_criacao,inserido_por,email,local,sabendo_de_nos_por,data_atualizacao,...,id_ibrati,email_corporativo,cargo_atual,projeto_atual,cliente,unidade,data_admissao,data_ultima_promocao,nome_superior_imediato,email_superior_imediato
0,31000,,(11) 97048-2708,,10-11-2021 07:29:49,Luna Correia,carolina_aparecida@gmail.com,,,10-11-2021 07:29:49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31001,,(11) 93723-4396,Analista Administrativo,10-11-2021 08:56:16,Laura Pacheco,eduardo_rios@hotmail.com,"São Paulo, São Paulo",Outros,11-11-2021 11:10:31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31002,,(11) 92399-9824,Administrativo | Financeiro,10-11-2021 09:01:00,Laura Pacheco,pedro_henrique_carvalho@gmail.com,"São Paulo, São Paulo",Anúncio,10-11-2021 11:42:36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31003,,(11) 98100-1727,Área administrativa,10-11-2021 09:08:13,Laura Pacheco,thiago_barbosa@hotmail.com,"São Paulo, São Paulo",Site de Empregos,10-11-2021 16:04:51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,31004,,(11) 92517-2678,,10-11-2021 09:18:46,Maria Clara Pires,diogo_das_neves@hotmail.com,,,10-11-2021 09:22:03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42477,5995,,(11) 92622-2088,Crm / Líder de Projetos,14-03-2018 16:35:14,pelo próprio candidato,gabrielly_novaes@gmail.com,,Outros,14-03-2018 16:35:14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42478,5996,,(41) 93009-5141,Crm,14-03-2018 16:38:28,pelo próprio candidato,sr._gabriel_rios@gmail.com,,Outros,14-03-2018 16:38:28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42479,5997,,(11) 97837-3941,Crm,14-03-2018 16:41:52,pelo próprio candidato,dr._mathias_da_cunha@gmail.com,,Outros,14-03-2018 16:41:52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42480,5998,,(61) 96806-8075,Crm,14-03-2018 16:42:49,pelo próprio candidato,bruna_vieira@hotmail.com,,Outros,14-03-2018 16:42:49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Write

### Write Functions

In [33]:
def write_json_file(filename: str, content : dict, base_path: str = SILVER_PATH, encoding: str = 'utf-8'):
    full_path = os.path.join(base_path, f'{filename}.json')

    with open(full_path, 'w+', encoding=encoding) as file:
        json.dump(content, file)

def read_json(filename, encoding: str = 'utf-8'):
    with open(filename, 'r+', encoding=encoding) as file:
        value = json.load(file)
    return value

### Write Execute

In [42]:
# Etapa 8: Salvar Parquet tratado
output_parquet_application = os.path.join(SILVER_PATH, "{filename}.{extension}")


df_applications.to_parquet(output_parquet_application.format(filename='application', extension='parquet'))
df_applications.to_csv(output_parquet_application.format(filename='application', extension='csv'))

In [43]:
df_test = pd.read_parquet(output_parquet_application.format(filename='application', extension='parquet'))
df_test

,job_id,telefone_recado,telefone,objetivo_profissional,data_criacao,inserido_por,email,local,sabendo_de_nos_por,data_atualizacao,...,id_ibrati,email_corporativo,cargo_atual,projeto_atual,cliente,unidade,data_admissao,data_ultima_promocao,nome_superior_imediato,email_superior_imediato
0,31000,,(11) 97048-2708,,10-11-2021 07:29:49,Luna Correia,carolina_aparecida@gmail.com,,,10-11-2021 07:29:49,...,None,None,None,None,None,None,None,None,None,None
1,31001,,(11) 93723-4396,Analista Administrativo,10-11-2021 08:56:16,Laura Pacheco,eduardo_rios@hotmail.com,"São Paulo, São Paulo",Outros,11-11-2021 11:10:31,...,None,None,None,None,None,None,None,None,None,None
2,31002,,(11) 92399-9824,Administrativo | Financeiro,10-11-2021 09:01:00,Laura Pacheco,pedro_henrique_carvalho@gmail.com,"São Paulo, São Paulo",Anúncio,10-11-2021 11:42:36,...,None,None,None,None,None,None,None,None,None,None
3,31003,,(11) 98100-1727,Área administrativa,10-11-2021 09:08:13,Laura Pacheco,thiago_barbosa@hotmail.com,"São Paulo, São Paulo",Site de Empregos,10-11-2021 16:04:51,...,None,None,None,None,None,None,None,None,None,None
4,31004,,(11) 92517-2678,,10-11-2021 09:18:46,Maria Clara Pires,diogo_das_neves@hotmail.com,,,10-11-2021 09:22:03,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42477,5995,,(11) 92622-2088,Crm / Líder de Projetos,14-03-2018 16:35:14,pelo próprio candidato,gabrielly_novaes@gmail.com,,Outros,14-03-2018 16:35:14,...,None,None,None,None,None,None,None,None,None,None
42478,5996,,(41) 93009-5141,Crm,14-03-2018 16:38:28,pelo próprio candidato,sr._gabriel_rios@gmail.com,,Outros,14-03-2018 16:38:28,...,None,None,None,None,None,None,None,None,None,None
42479,5997,,(11) 97837-3941,Crm,14-03-2018 16:41:52,pelo próprio candidato,dr._mathias_da_cunha@gmail.com,,Outros,14-03-2018 16:41:52,...,None,None,None,None,None,None,None,None,None,None
42480,5998,,(61) 96806-8075,Crm,14-03-2018 16:42:49,pelo próprio candidato,bruna_vieira@hotmail.com,,Outros,14-03-2018 16:42:49,...,None,None,None,None,None,None,None,None,None,None
